In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
train_dir = "../data/ultrasound/train"
val_dir = "../data/ultrasound/val"

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

train_generator.class_indices

Found 1578 images belonging to 3 classes.
Found 1578 images belonging to 3 classes.


{'BENIGN': 0, 'MALIGNANT': 1, 'NORMAL': 2}

In [4]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False  # freeze backbone

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)

# Binary output
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

In [5]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
99/99 [==============================] - 112s 1s/step - loss: 0.6505 - accuracy: 0.3809 - val_loss: 0.4670 - val_accuracy: 0.4043
Epoch 2/10
99/99 [==============================] - 109s 1s/step - loss: 0.5480 - accuracy: 0.4183 - val_loss: 0.4102 - val_accuracy: 0.5177
Epoch 3/10
99/99 [==============================] - 106s 1s/step - loss: 0.4467 - accuracy: 0.4480 - val_loss: 0.2916 - val_accuracy: 0.4594
Epoch 4/10
99/99 [==============================] - 103s 1s/step - loss: 0.3437 - accuracy: 0.4854 - val_loss: 0.2549 - val_accuracy: 0.6077
Epoch 5/10
99/99 [==============================] - 103s 1s/step - loss: 0.2415 - accuracy: 0.5013 - val_loss: 0.1800 - val_accuracy: 0.4550
Epoch 6/10
99/99 [==============================] - 105s 1s/step - loss: 0.1883 - accuracy: 0.5101 - val_loss: 0.1086 - val_accuracy: 0.4550
Epoch 7/10
99/99 [==============================] - 113s 1s/step - loss: 0.1316 - accuracy: 0.5317 - val_loss: -0.0833 - val_accuracy: 0.6153
Epoch 8/10
9

In [7]:
model.save("../model/ultrasound_breast_model.h5")

C:\Users\ASUS\OneDrive\Desktop\MedLens\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
